## 1.  Scrap content from wiki page

In [1]:
#Import necessary packages
import pandas as pd
!pip install bs4
!pip install lxml
from bs4 import BeautifulSoup
import requests
import numpy as np
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import folium # map rendering library


# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

     |████████████████████████████████| 122kB 4.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.5MB 6.3MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarc

Scrape the "raw" table

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

soup = BeautifulSoup(source,  "html.parser")
table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]

#toronto_dropNotAssigned = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
 # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Remove "\n" at the end of each string in the all the columns

In [3]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Group all neighborhoods with the same postal code

In [33]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


Printing number of rows of in my dataframe

In [4]:
print("Shape: ", df.shape)

Shape:  (180, 3)


## 2.  Get the latitude and the longitude coordinates of each neighborhood

The given csv file will be used

In [5]:
filename = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(filename)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


2 dataframes "df" and "df_coordinates" will be merged into one dataframe.

In [6]:
df_toronto = pd.merge(df, df_coordinates, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto = df_toronto.dropna()
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 3.  Explore and cluster the neighborhoods in Toronto

3.1. Get the latitude and longitude values of Toronto

In [7]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


3.2. Create a map of the whole Toronto City with neighborhoods superimposed on top.

In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

Add markers to the map.

In [9]:
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

## 3.3. Map of a part of Toronto City¶

Only the boroughs that contain the word "Toronto" will be used

In [10]:
df_toronto_area = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_area.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Plot again the map and the markers for this region.

In [11]:
map_toronto_area = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_area['Latitude'], 
        df_toronto_area['Longitude'], 
        df_toronto_area['Borough'], 
        df_toronto_area['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_area)  

map_toronto_area

## 3.4. Explore the first neighborhood in our data frame "df_toronto"

In [12]:
neighborhood_name = df_toronto_area.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'Regent Park, Harbourfront'.


Get the neighborhood's latitude and longitude values.

In [13]:
neighborhood_latitude = df_toronto_area.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_area.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [ ]:
Now, let's get the top 100 venues that are in Regent Park, Harbourfron within a radius of 1500 meters.

In [95]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20201222'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [32]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?ll=43.67635739999999,-79.2930312&client_id=KKIOXEJBYPET2JDYCHUBCDHAOFNDKCPZ41NN4WQUVUEVB0O4&client_secret=PMHAPJJTJHHP00HAJN0Y2OIJQSHAJNBQ2NHHEW4IMTNRJNNC&v=20201222&radius=1500&limit=100'
url


'https://api.foursquare.com/v2/venues/explore?ll=43.67635739999999,-79.2930312&client_id=KKIOXEJBYPET2JDYCHUBCDHAOFNDKCPZ41NN4WQUVUEVB0O4&client_secret=PMHAPJJTJHHP00HAJN0Y2OIJQSHAJNBQ2NHHEW4IMTNRJNNC&v=20201222&radius=1500&limit=100'

In [33]:
 #get the result to a json file
results = requests.get(url).json()

Function that extracts the category of the venue

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now clean the json and structure it into a pandas dataframe.

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
2,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272
3,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993
4,The Beech Tree,Gastropub,43.680493,-79.288846
...,...,...,...,...
95,Maple Leaf Dairy Ice Cream Parlor,Ice Cream Shop,43.689677,-79.294779
96,The Salty Dog,Restaurant,43.669944,-79.300894
97,Grillies,Mexican Restaurant,43.688550,-79.300025
98,Scarboro Beach,Beach,43.669273,-79.293902


In [66]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 3.5 Explore neighborhoods in a part of Toronto City

Function to repeat the same process to all the neighborhoods in Area of Toronto.

In [77]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url =  'https://api.foursquare.com/v2/venues/explore?ll=43.67635739999999,-79.2930312&client_id=KKIOXEJBYPET2JDYCHUBCDHAOFNDKCPZ41NN4WQUVUEVB0O4&client_secret=PMHAPJJTJHHP00HAJN0Y2OIJQSHAJNBQ2NHHEW4IMTNRJNNC&v=20201218&radius=1500&limit=100'
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_area_venues

In [78]:
toronto_area_venues = getNearbyVenues(names=df_toronto_area['Neighborhood'],
                                   latitudes=df_toronto_area['Latitude'],
                                   longitudes=df_toronto_area['Longitude']
                                  )

In [80]:
#toronto_area_venues.head()

print(toronto_area_venues.shape)
toronto_area_venues.head()

(3900, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Glen Manor Ravine,43.676821,-79.293942,Trail
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,"Regent Park, Harbourfront",43.65426,-79.360636,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,"Regent Park, Harbourfront",43.65426,-79.360636,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,"Regent Park, Harbourfront",43.65426,-79.360636,The Beech Tree,43.680493,-79.288846,Gastropub


Let's check how many venues were returned for each neighborhood.

In [81]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Let's find out how many unique categories can be curated from all the returned venues

In [82]:
print('There are {} uniques categories.'.format(len(toronto_area_venues['Venue Category'].unique())))

There are 55 uniques categories.


## 3.6  Analyze Each Neighborhood

In [83]:
# one hot encoding
toronto_area_onehot = pd.get_dummies(toronto_area_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_area_onehot['Neighborhood'] = toronto_area_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_area_onehot.columns[-1]] + list(toronto_area_onehot.columns[:-1])
toronto_area_onehot = toronto_area_onehot[fixed_columns]

toronto_area_onehot.head()

,Neighborhood,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Store,Bookstore,...,Restaurant,Sandwich Place,Skating Rink,Sushi Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [84]:
toronto_area_grouped = toronto_area_onehot.groupby('Neighborhood').mean().reset_index()
toronto_area_grouped.head()

,Neighborhood,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Store,Bookstore,...,Restaurant,Sandwich Place,Skating Rink,Sushi Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant
0,Berczy Park,0.01,0.03,0.01,0.03,0.02,0.02,0.04,0.01,0.01,...,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
1,"Brockton, Parkdale Village, Exhibition Place",0.01,0.03,0.01,0.03,0.02,0.02,0.04,0.01,0.01,...,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
2,"Business reply mail Processing Centre, South C...",0.01,0.03,0.01,0.03,0.02,0.02,0.04,0.01,0.01,...,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.01,0.03,0.01,0.03,0.02,0.02,0.04,0.01,0.01,...,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
4,Central Bay Street,0.01,0.03,0.01,0.03,0.02,0.02,0.04,0.01,0.01,...,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


Check the 10 most common venues in each neighborhood.

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_area_grouped['Neighborhood']

for ind in np.arange(toronto_area_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_area_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
4,Central Bay Street,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place


## 3.7 Cluster neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_area_merged = df_toronto_area

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_area_merged = toronto_area_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_area_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place


Finally, let's visualize the resulting clusters

In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_area_merged['Latitude'], 
        toronto_area_merged['Longitude'], 
        toronto_area_merged['Neighborhood'], 
        toronto_area_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3.8. Examine Clusters¶

Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Cluster 1

In [88]:
toronto_area_merged.loc[toronto_area_merged['Cluster Labels'] == 0, toronto_area_merged.columns[[1] + list(range(5, toronto_area_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
1,Downtown Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
2,Downtown Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
3,Downtown Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
4,East Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
5,Downtown Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
6,Downtown Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
7,Downtown Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
8,Downtown Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place
9,West Toronto,0,Coffee Shop,Pub,Grocery Store,Beach,Japanese Restaurant,Breakfast Spot,BBQ Joint,Bakery,Ice Cream Shop,Sandwich Place


Cluster 2

In [89]:
toronto_area_merged.loc[toronto_area_merged['Cluster Labels'] == 1, toronto_area_merged.columns[[1] + list(range(5, toronto_area_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Cluster 3

In [61]:
toronto_area_merged.loc[toronto_area_merged['Cluster Labels'] == 2, toronto_area_merged.columns[[1] + list(range(5, toronto_area_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Cluster 4

In [93]:
toronto_area_merged.loc[toronto_area_merged['Cluster Labels'] == 3, toronto_area_merged.columns[[1] + list(range(5, toronto_area_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Cluster 5

In [94]:
toronto_area_merged.loc[toronto_area_merged['Cluster Labels'] == 4, toronto_area_merged.columns[[1] + list(range(5, toronto_area_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
